In [1]:
!pip install pyreadstat==1.2.0 pandas==1.4.4 scipy==1.10.1 plotly==5.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from collections import defaultdict
import math 
import os

import numpy as np
import pyreadstat
import pandas as pd
from scipy.stats import chisquare
import plotly.express as px

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Данные Комплексного обследования условий жизни населения по 2020 году. Файл по домохозяйствам

In [4]:
fpath = 'drive/MyDrive/коуж/HHOLD_OSN_2020.sav' # Источник данных: https://gks.ru .

In [5]:
df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath, num_processes=16, encoding='cp1251') 
df.head()

,H00_02,H00_04,H00_06,H00_07,H00_08,H00_09,H00_10,H00_11,H00_12,H00_13,...,PM_DX_MES,PM_DX_SRED_MES,MALOIM,KVZV,SUPR_PAR,DX,V407,GL_DH_POL,GL_DH_VOZR,GL_DH_LET
0,46,2.0,37419.0,9.0,22.0,10.0,2020.0,13.0,0.0,13.0,...,20206.0,10103.0,0.0,575.713802,1.0,1.0,NaN,1.0,2.0,70.0
1,46,2.0,37702.0,9.0,22.0,10.0,2020.0,15.0,0.0,15.0,...,58140.0,14535.0,0.0,1025.493642,1.0,NaN,NaN,1.0,1.0,50.0
2,46,2.0,37703.0,9.0,23.0,10.0,2020.0,11.0,0.0,11.0,...,58140.0,14535.0,1.0,882.435870,1.0,NaN,NaN,1.0,1.0,30.0
3,46,2.0,37073.0,9.0,23.0,10.0,2020.0,17.0,0.0,17.0,...,10103.0,10103.0,0.0,525.487031,NaN,1.0,NaN,2.0,2.0,69.0
4,46,2.0,37074.0,9.0,24.0,10.0,2020.0,10.0,0.0,10.0,...,14941.0,14941.0,0.0,757.206120,NaN,NaN,NaN,1.0,1.0,56.0


Что такое частота - читаем тут (да, это сайт с оформлением из нулевых и смешной рекламой, но зато всё понятно): http://mathprofi.ru/matematicheskaya_statistika.html

In [6]:
columns_to_count_frequencies = [
    'H02_13_01_01', # Наличие лифта
    'H02_13_02_01', # Наличие мусоропровода
    'H02_18', # Вид водопровода в доме
    'H02_221_01', # Наличие стиральной машины
    'H02_221_02', # Наличие холодильника
    'H02_23', # Пользование дома бытовым газом
    'H02_27', # Тип горячего водоснабжения 
    'H02_29', # Тип отопления в доме
    'H02_10', # Оценка состояния жилого помещения
    'H02_01', # Условия проживания домохозяйства
    'H02_03', # Является ли основное жилье собственностью домохозяйства
    ]

документация pandas - есть наглядный пример разницы между частотами и частостями (у них они называются relative frequencies)
https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html


In [7]:
def frequencuies_pandas(dataframe, columns_to_count_frequencies, factors, name):
    """
    Функция, которая считает частоты (абсолютные) и частости (относительные частоты) ответов на вопросы и
    рисует "тортовые" графики (круговые диаграммы, пироги) какая доля респондентов на какой вопрос как ответила

    dataframe (pd.DataFrame) - таблица с вопросами и ответами
    columns_to_count_frequencies (list) - вопросы, для которых считать частоты и строить графики
    factors (defaultdict) - словарь, где ключи это вопросы, а значения - списки с частотами ответов
    name (str) - как назвать таблицу (заголовок верхнего уровня)
    """
    for column in columns_to_count_frequencies:
        try:
            question = meta.column_names_to_labels[column]
            print(question)
            frequencies = dataframe[column].value_counts(dropna=False)
            # value_counts возвращает Series, где value это индекс, а значения - частоты или частости
            frequencies_rel = dataframe[column].value_counts(normalize=True, dropna=False)

            tuples = [(name, 'Абс.'), (name, 'Отн.')]
            index = pd.MultiIndex.from_tuples(tuples) # мультииндекс нужен, чтобы столбцы могли иметь заголовки разного уровня, и чтобы заголовок верхнего уровня мог объединять несколько столбцов
            question_dict = {'Абс.': frequencies, 'Отн.':frequencies_rel}
            question_df = pd.DataFrame(question_dict)
            
            question_df.index = pd.Series(question_df.index).apply(lambda x: meta.value_labels[meta.variable_to_label[column]].get(x))
            fig = px.pie(question_df, values='Абс.', names=question_df.index, title=question)
            fig.show()
            total = {'Абс.': question_df['Абс.'].sum(), 'Отн.': question_df['Отн.'].sum()}
            new_row = pd.DataFrame(total, index=['Total'])
            question_df = pd.concat([question_df, new_row], axis=0)

            question_df.columns = index
            print(question_df)
            factors[question].append(question_df)
        except KeyError: # обработка ошибки: если нет метки spss для столбца, выругиваемся и переходим к следующему столбцу
            print(f"{column} отсутствует в meta")
        print('-------------------------------------------------------------------')

In [8]:
# заведем словарь factors, где ключами будут вопросы (например, наличие лифта?),
# а значениями - списки таблиц с частотами разных ответов на эти вопросы по всей РФ, 
# только сельской местности во всей РФ, 
# только  городской местности во всей РФ, 
# только сельской местности в Якутии, 
# только  городской местности в Якутии,
# чтобы не прописывать заранее все ключи с пустыми списками в значениях, 
# используем структуру данных defaultdict https://pythonpip.ru/osnovy/defaultdict-v-python-klass-slovarya-modulya-collections
factors = defaultdict(list) 
frequencuies_pandas(df, columns_to_count_frequencies, factors, 'Вся РФ')
print('===================================================================')

Наличие лифта


      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Вся РФ          
        Абс.      Отн.
Нет    28051  0.467563
None   20905  0.348452
Да     11038  0.183985
Total  59994  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   Вся РФ          
                                                     Абс.      Отн.
Централизованный водопровод                         51702  0.861786
Водопровод отсутствует                               3287  0.054789
Водопровод из индивидуальной артезианской скважины   2909  0.048488
Водопровод из колодца                                2079  0.034653
None                                                   17  0.000283
Total                                               59994  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Вся РФ          
                                           Абс.      Отн.
Да и имеется и используется               58290  0.971597
Нет, не имеется, не можем себе позволить    635  0.010584
Да, имеется, но не используется             406  0.006767
Нет, не имею по другим причинам             331  0.005517
Нет, не имеется, нет необходимости          315  0.005251
None                                         17  0.000283
Total                                     59994  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   Вся РФ          
                                                     Абс.      Отн.
Центральное горячее водоснабжение                   28354  0.472614
Горячее водоснабжение от местных (индивидуальны...  23459  0.391022
Горячее водоснабжение отсутствует                    8164  0.136080
None                                                   17  0.000283
Total                                               59994  1.000000
-------------------------------------------------------------------
Тип отопления в доме


                                              Вся РФ          
                                                Абс.      Отн.
Центральное отопление                          36081  0.601410
Отопление от индивидуальных установок, котлов  17574  0.292929
Печное отопление                                6165  0.102760
Другой способ отопления                          157  0.002617
None                                              17  0.000283
Total                                          59994  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Вся РФ          
                     Абс.      Отн.
Удовлетворительное  27005  0.450128
Хорошее             26372  0.439577
Отличное             4039  0.067323
Плохое               2343  0.039054
Очень плохое          235  0.003917
Total               59994  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Вся РФ          
                                               Абс.      Отн.
Отдельная квартира                            38786  0.646498
Индивидуальный (одноквартирный) дом           17711  0.295213
Часть индивидуального (одноквартирного) дома   3177  0.052955
Коммунальная квартира                           251  0.004184
Общежитие                                        53  0.000883
Другое                                           16  0.000267
Total                                         59994  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Вся РФ          
                                                     Абс.      Отн.
Да, без использования ипотечных средств или ипо...  50710  0.845251
Нет                                                  6162  0.102710
Да, с использованием ипотечных средств (ипотека...   3053  0.050888
None                                                   69  0.001150
Total                                               59994  1.000000
-------------------------------------------------------------------


In [9]:
def print_first_key_value_in_dict(the_dict):
    """Распечатать первую пару ключ-значение в словареподобном объекте(dict, defaultdict...)"""
    for k, v in the_dict.items():
        print(k)
        print(v)
        break

In [10]:
# после каждого вызова функции frequencuies_pandas будем проверять, что в словаре factors лежит то, что нужно
print_first_key_value_in_dict(factors)

Наличие лифта
[      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000]


In [11]:
# и так же, как  в ноутбуке descriptive_statistics, выбираем городскую и сельскую местность
for name in df['R_1'].unique():
    selected = df[df['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, name)
    print('===================================================================')

Наличие лифта


      сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5331  0.288661
Да                   101  0.005469
Total              18468  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5325  0.288337
Да                   107  0.005794
Total              18468  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   сельская местность  \
                                                                 Абс.   
Централизованный водопровод                                     12813   
Водопровод отсутствует                                           2324   
Водопровод из индивидуальной артезианской скважины               1947   
Водопровод из колодца                                            1376   
None                                                                8   
Total                                                           18468   

                                                              
                                                        Отн.  
Централизованный водопровод                         0.693795  
Водопровод отсутствует                              0.125839  
Водопровод из индивидуальной артезианской скважины  0.105426  
Водопровод из колодца                               0.074507  
None                                 

                                         сельская местность          
                                                       Абс.      Отн.
Да и имеется и используется                           17516  0.948451
Нет, не имеется, не можем себе позволить                366  0.019818
Да, имеется, но не используется                         232  0.012562
Нет, не имею по другим причинам                         185  0.010017
Нет, не имеется, нет необходимости                      161  0.008718
None                                                      8  0.000433
Total                                                 18468  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   городская местность  \
                                                                  Абс.   
Центральное горячее водоснабжение                                26317   
Горячее водоснабжение от местных (индивидуальны...               12465   
Горячее водоснабжение отсутствует                                 2735   
None                                                                 9   
Total                                                            41526   

                                                              
                                                        Отн.  
Центральное горячее водоснабжение                   0.633748  
Горячее водоснабжение от местных (индивидуальны...  0.300173  
Горячее водоснабжение отсутствует                   0.065862  
None                                                0.000217  
Total                                               1.000000  
----------------------------------------

                                              городская местность          
                                                             Абс.      Отн.
Центральное отопление                                       32216  0.775803
Отопление от индивидуальных установок, котлов                7392  0.178009
Печное отопление                                             1872  0.045080
Другой способ отопления                                        37  0.000891
None                                                            9  0.000217
Total                                                       41526  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   городская местность          
                                  Абс.      Отн.
Хорошее                          19030  0.458267
Удовлетворительное               18072  0.435197
Отличное                          3013  0.072557
Плохое                            1274  0.030680
Очень плохое                       137  0.003299
Total                            41526  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             городская местность          
                                                            Абс.      Отн.
Отдельная квартира                                         33420  0.804797
Индивидуальный (одноквартирный) дом                         6859  0.165174
Часть индивидуального (одноквартирного) дома                1001  0.024105
Коммунальная квартира                                        198  0.004768
Общежитие                                                     40  0.000963
Другое                                                         8  0.000193
Total                                                      41526  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   городская местность  \
                                                                  Абс.   
Да, без использования ипотечных средств или ипо...               34672   
Нет                                                               4179   
Да, с использованием ипотечных средств (ипотека...                2627   
None                                                                48   
Total                                                            41526   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.834947  
Нет                                                 0.100636  
Да, с использованием ипотечных средств (ипотека...  0.063262  
None                                                0.001156  
Total                                               1.000000  
----------------------------------------

In [12]:
print_first_key_value_in_dict(factors)

Наличие лифта
[      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000,       сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5331  0.288661
Да                   101  0.005469
Total              18468  1.000000,       городская местность          
                     Абс.      Отн.
Нет                 21256  0.511872
Да                  12401  0.298632
None                 7869  0.189496
Total               41526  1.000000]


In [13]:
region = df[df['H00_02']=='98'] # маской выбираем по коду субъекта только cвой регион, как делали в ноутбуке descriptive_statistics
#В этих данных 98 это строка, а не число. Если без кавычек, то ничего не найдется.


In [14]:
region_name = 'Якутия'

In [15]:
region.head()

,H00_02,H00_04,H00_06,H00_07,H00_08,H00_09,H00_10,H00_11,H00_12,H00_13,...,PM_DX_MES,PM_DX_SRED_MES,MALOIM,KVZV,SUPR_PAR,DX,V407,GL_DH_POL,GL_DH_VOZR,GL_DH_LET
999,98,1.0,70348.0,4.0,24.0,10.0,2020.0,15.0,40.0,16.0,...,18728.0,18728.000000,0.0,407.593312,NaN,NaN,NaN,2.0,1.0,50.0
1000,98,1.0,70364.0,4.0,21.0,10.0,2020.0,15.0,36.0,16.0,...,18728.0,18728.000000,0.0,407.593312,NaN,NaN,NaN,2.0,1.0,50.0
1001,98,1.0,70362.0,4.0,25.0,10.0,2020.0,12.0,30.0,13.0,...,18728.0,18728.000000,1.0,407.593312,NaN,NaN,NaN,2.0,1.0,25.0
1002,98,1.0,70465.0,4.0,24.0,10.0,2020.0,17.0,35.0,18.0,...,37456.0,18728.000000,0.0,1117.087036,1.0,NaN,NaN,1.0,1.0,52.0
1003,98,1.0,70577.0,4.0,1.0,11.0,2020.0,16.0,0.0,16.0,...,55846.0,18615.333333,1.0,954.453765,NaN,NaN,NaN,2.0,1.0,38.0


In [16]:
# и так же, как  в ноутбуке descriptive_statistics, выбираем городскую и сельскую местность
for name in region['R_1'].unique():
    print(name)
    selected = region[region['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    
    print(selected)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f'{region_name} {name}')
    print('===================================================================')

1.0
      H00_02  H00_04   H00_06  H00_07  H00_08  H00_09  H00_10  H00_11  H00_12  \
999       98     1.0  70348.0     4.0    24.0    10.0  2020.0    15.0    40.0   
1000      98     1.0  70364.0     4.0    21.0    10.0  2020.0    15.0    36.0   
1001      98     1.0  70362.0     4.0    25.0    10.0  2020.0    12.0    30.0   
1002      98     1.0  70465.0     4.0    24.0    10.0  2020.0    17.0    35.0   
1003      98     1.0  70577.0     4.0     1.0    11.0  2020.0    16.0     0.0   
...      ...     ...      ...     ...     ...     ...     ...     ...     ...   
52564     98     1.0  70390.0     1.0     1.0    11.0  2020.0    12.0    22.0   
52565     98     1.0  70506.0     1.0    31.0    10.0  2020.0    14.0    55.0   
52566     98     1.0  70389.0     1.0    31.0    10.0  2020.0    16.0     8.0   
52567     98     1.0  70602.0     1.0     1.0    11.0  2020.0    13.0    12.0   
52568     98     1.0  70635.0     1.0     1.0    11.0  2020.0    15.0     1.0   

       H00_13  ...  PM_

      Якутия городская местность          
                            Абс.      Отн.
Нет                          219  0.737374
Да                            56  0.188552
None                          22  0.074074
Total                        297  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городская местность          
                            Абс.      Отн.
Нет                          220  0.740741
Да                            55  0.185185
None                          22  0.074074
Total                        297  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городская местность          
                                                  Абс.      Отн.
Централизованный водопровод                        273  0.919192
Водопровод отсутствует                              23  0.077441
Водопровод из колодца                                1  0.003367
Total                                              297  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Якутия городская местность          
                                                               Абс.      Отн.
Да и имеется и используется                                     292  0.983165
Нет, не имеется, не можем себе позволить                          3  0.010101
Нет, не имею по другим причинам                                   2  0.006734
Total                                                           297  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   Якутия сельская местность  \
                                                                        Абс.   
Горячее водоснабжение отсутствует                                        141   
Центральное горячее водоснабжение                                         33   
Горячее водоснабжение от местных (индивидуальны...                        15   
Total                                                                    189   

                                                              
                                                        Отн.  
Горячее водоснабжение отсутствует                   0.746032  
Центральное горячее водоснабжение                   0.174603  
Горячее водоснабжение от местных (индивидуальны...  0.079365  
Total                                               1.000000  
-------------------------------------------------------------------
Тип отопления в доме


                                              Якутия сельская местность  \
                                                                   Абс.   
Печное отопление                                                     73   
Отопление от индивидуальных установок, котлов                        60   
Центральное отопление                                                47   
Другой способ отопления                                               9   
Total                                                               189   

                                                         
                                                   Отн.  
Печное отопление                               0.386243  
Отопление от индивидуальных установок, котлов  0.317460  
Центральное отопление                          0.248677  
Другой способ отопления                        0.047619  
Total                                          1.000000  
-------------------------------------------------------------------


                   Якутия сельская местность          
                                        Абс.      Отн.
Удовлетворительное                        88  0.465608
Хорошее                                   62  0.328042
Отличное                                  18  0.095238
Плохое                                    16  0.084656
Очень плохое                               5  0.026455
Total                                    189  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Якутия сельская местность  \
                                                                  Абс.   
Индивидуальный (одноквартирный) дом                                160   
Отдельная квартира                                                  21   
Часть индивидуального (одноквартирного) дома                         8   
Total                                                              189   

                                                        
                                                  Отн.  
Индивидуальный (одноквартирный) дом           0.846561  
Отдельная квартира                            0.111111  
Часть индивидуального (одноквартирного) дома  0.042328  
Total                                         1.000000  
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельская местность  \
                                                                        Абс.   
Да, без использования ипотечных средств или ипо...                       140   
Нет                                                                       44   
Да, с использованием ипотечных средств (ипотека...                         5   
Total                                                                    189   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.740741  
Нет                                                 0.232804  
Да, с использованием ипотечных средств (ипотека...  0.026455  
Total                                               1.000000  
-------------------------------------------------------------------


In [17]:
print_first_key_value_in_dict(factors)

Наличие лифта
[      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000,       сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5331  0.288661
Да                   101  0.005469
Total              18468  1.000000,       городская местность          
                     Абс.      Отн.
Нет                 21256  0.511872
Да                  12401  0.298632
None                 7869  0.189496
Total               41526  1.000000,       Якутия городская местность          
                            Абс.      Отн.
Нет                          219  0.737374
Да                            56  0.188552
None                          22  0.074074
Total                        297  1.000000,       Якутия сельская местность          
                           Абс.      Отн.
None                        168  0.888889
Нет                          21  

In [18]:
# выбираем тип города
for name in region['R_1_1'].unique():
    selected = region[region['R_1_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1_1']].get(name)
    
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f"{region_name} {name}")
    print('===================================================================')

Наличие лифта


      Якутия городской,  250,0 - 499,9 тыс.чел          
                                          Абс.      Отн.
Нет                                         53  0.654321
Да                                          28  0.345679
Total                                       81  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городской,  250,0 - 499,9 тыс.чел     
                                          Абс. Отн.
Нет                                         81  1.0
Total                                       81  1.0
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городской,  250,0 - 499,9 тыс.чел     
                                                                Абс. Отн.
Централизованный водопровод                                       81  1.0
Total                                                             81  1.0
-------------------------------------------------------------------
Наличие стиральной машины


                                Якутия городской,  250,0 - 499,9 тыс.чел  \
                                                                    Абс.   
Да и имеется и используется                                           79   
Нет, не имею по другим причинам                                        2   
Total                                                                 81   

                                           
                                     Отн.  
Да и имеется и используется      0.975309  
Нет, не имею по другим причинам  0.024691  
Total                            1.000000  
-------------------------------------------------------------------
Наличие холодильника


...

                                                   Якутия сельский,  более 5000 чел.  \
                                                                                Абс.   
Горячее водоснабжение отсутствует                                                 24   
Горячее водоснабжение от местных (индивидуальны...                                 3   
Total                                                                             27   

                                                              
                                                        Отн.  
Горячее водоснабжение отсутствует                   0.888889  
Горячее водоснабжение от местных (индивидуальны...  0.111111  
Total                                               1.000000  
-------------------------------------------------------------------
Тип отопления в доме


                        Якутия сельский,  более 5000 чел.          
                                                     Абс.      Отн.
Печное отопление                                       18  0.666667
Другой способ отопления                                 9  0.333333
Total                                                  27  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Якутия сельский,  более 5000 чел.          
                                                Абс.      Отн.
Хорошее                                           15  0.555556
Удовлетворительное                                 9  0.333333
Отличное                                           3  0.111111
Total                                             27  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                    Якутия сельский,  более 5000 чел.     
                                                                 Абс. Отн.
Индивидуальный (одноквартирный) дом                                27  1.0
Total                                                              27  1.0
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельский,  более 5000 чел.  \
                                                                                Абс.   
Да, без использования ипотечных средств или ипо...                                24   
Нет                                                                                2   
Да, с использованием ипотечных средств (ипотека...                                 1   
Total                                                                             27   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.888889  
Нет                                                 0.074074  
Да, с использованием ипотечных средств (ипотека...  0.037037  
Total                                               1.000000  
-------------------------------------------------------------------


In [19]:
print_first_key_value_in_dict(factors)

Наличие лифта
[      Вся РФ          
        Абс.      Отн.
Нет    26587  0.443161
None   20905  0.348452
Да     12502  0.208388
Total  59994  1.000000,       сельская местность          
                    Абс.      Отн.
None               13036  0.705870
Нет                 5331  0.288661
Да                   101  0.005469
Total              18468  1.000000,       городская местность          
                     Абс.      Отн.
Нет                 21256  0.511872
Да                  12401  0.298632
None                 7869  0.189496
Total               41526  1.000000,       Якутия городская местность          
                            Абс.      Отн.
Нет                          219  0.737374
Да                            56  0.188552
None                          22  0.074074
Total                        297  1.000000,       Якутия сельская местность          
                           Абс.      Отн.
None                        168  0.888889
Нет                          21  

### Критерии существенности различий и Хи-квадрат

In [21]:
questions = []
for question_name, answers in factors.items():
    print(question_name)
    question = pd.concat(answers, axis=1).iloc[:,:12]
    questions.append((question_name, question))
    # следующие три строчки можно закомментировать, если не нужны сохраненные таблицы. папку tables надо создать на диске заранее
    if os.path.isdir('drive/MyDrive/tables/'):
        table_by_questions_path = f'drive/MyDrive/tables/{question_name}.xlsx'
        question.to_excel(table_by_questions_path)

Наличие лифта
Наличие мусоропровода
Вид водопровода в доме
Наличие стиральной машины
Наличие холодильника
Пользование дома бытовым газом
Тип горячего водоснабжения
Тип отопления в доме
Оценка состояния жилого помещения
Условия проживания домохозяйства
Является ли основное жилье собственностью домохозяйства


In [22]:
def criteria(questions, area_1, area_2, area_3):
    for question in questions:
        print(f"Вопрос {question[0]}")
        current_question = question[1]
        
        ### Критерий существенности различия
        omegas_1 = current_question[area_1, 'Отн.'][:-1]
        omegas_2 = current_question[area_2, 'Отн.'][:-1]
        # в этих данных порядки n это десятки тысяч, и даже различия частостей на тысячные доли могут оказаться значимыми
        # можно попробовать поставить n меньше, например, 100, и увидеть, что значимых различий становится меньше 
        n1 = current_question[area_1, 'Абс.']['Total']
        n2 = current_question[area_2, 'Абс.']['Total']
        for response, o1, o2 in zip(current_question[:-1].index, omegas_1, omegas_2):
            print(f"Значение {response}")
            print(o1, o2)
            if np.isnan(o1) or np.isnan(o2):
                print("Ответ с данным значением не выбрал никто в сельской или городской местности. Подсчитать критерий существенности различия нельзя")
                continue
            print(n1, n2)
            try:
                criterion = abs(o1-o2)/math.sqrt((o1*(1-o1)/n1 + o2*(1-o2)/n2))
            except ZeroDivisionError:
                print("Деление на 0. Подсчитать критерий существенности различия для данного значения нельзя")
                continue
            print(f"Критерий существенности различия: {criterion}")
            print('--------------------------------')
        ### Критерий Хи-квадрат
        # чтобы тест был применим, должна быть одинаковая сумма наблюдений в обоих рядах
        # поэтому берем нормализованные значения, а затем умножаем на 1000, чтобы числа не были маленькие
        print(f"Хи-квадрат, {area_3} и {area_2}")
        chi = chisquare(current_question[area_2, 'Отн.'][:-1].apply(lambda x: x*1000), f_exp=current_question[area_3, 'Отн.'][:-1].apply(lambda x: x*1000), ddof=0)
        print(chi)
        print(f"Количество степеней свободы {current_question[area_2, 'Отн.'][:-1].shape[0] - 1}") # количество ответов на вопрос - 1
        print(f"Хи-квадрат, {area_3} и {area_1}")
        chi = chisquare(current_question[area_1, 'Отн.'][:-1].apply(lambda x: x*1000), f_exp=current_question[area_3, 'Отн.'][:-1].apply(lambda x: x*1000), ddof=0, axis=0)
        print(chi)
        print('---------------------------------------------------------------------')

Расчёт критерия существенности различий и критерия Хи-квадрат по каждой переменной для городской и сельской местности в пределах всей РФ:

In [23]:
print("Сельская и городская местность, вся РФ")
criteria(questions, 'сельская местность', 'городская местность', 'Вся РФ') # 'Вся РФ' - и сельская, и городская

Сельская и городская местность, вся РФ
Вопрос Наличие лифта
Значение Нет
0.2886614684860299 0.5118720801425612
18468 41526
Критерий существенности различия: 53.92220278883004
--------------------------------
Значение None
0.7058696123023609 0.18949573761017194
18468 41526
Критерий существенности различия: 133.59193082967084
--------------------------------
Значение Да
0.00546891921160927 0.2986321822472668
18468 41526
Критерий существенности различия: 126.88361228032454
--------------------------------
Хи-квадрат, Вся РФ и городская местность
Power_divergenceResult(statistic=122.24708802042366, pvalue=2.8469649602834148e-27)
Количество степеней свободы 2
Хи-квадрат, Вся РФ и сельская местность
Power_divergenceResult(statistic=618.0718872374902, pvalue=6.12907557684304e-135)
---------------------------------------------------------------------
Вопрос Наличие мусоропровода
Значение Нет
0.2883365821962313 0.5472715888840727
18468 41526
Критерий существенности различия: 62.658214623932814


Расчёт критерия существенности различий и критерия Хи-квадрат по каждой переменной для городской и сельской местности в пределах выбранного региона

In [24]:
print(f"Сельская и городская местность, {region_name}")
criteria(questions, f'{region_name} сельская местность', f'{region_name} городская местность', 'Вся РФ')

Сельская и городская местность, Якутия
Вопрос Наличие лифта
Значение Нет
0.1111111111111111 0.7373737373737373
189.0 297
Критерий существенности различия: 18.273060445080933
--------------------------------
Значение None
0.8888888888888888 0.07407407407407407
189.0 297
Критерий существенности различия: 29.68362070239246
--------------------------------
Значение Да
nan 0.18855218855218855
Ответ с данным значением не выбрал никто в сельской или городской местности. Подсчитать критерий существенности различия нельзя
Хи-квадрат, Вся РФ и Якутия городская местность
Power_divergenceResult(statistic=413.2647421903515, pvalue=1.8226423234659266e-90)
Количество степеней свободы 2
Хи-квадрат, Вся РФ и Якутия сельская местность
Power_divergenceResult(statistic=nan, pvalue=nan)
---------------------------------------------------------------------
Вопрос Наличие мусоропровода
Значение Нет
0.1111111111111111 0.7407407407407407
189.0 297
Критерий существенности различия: 18.413837543883965
----------

## Данные Комплексного обследования условий жизни населения по 2018 году. Файл по домохозяйствам

In [25]:
fpath = 'drive/MyDrive/коуж/HHOLD_2018.sav' # Источник данных: https://gks.ru .
df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath, num_processes=16, encoding='cp1251') 
df.head()

,H00_07,H00_09,H00_10,H00_11,H00_12,H00_13,H00_14,H01_15,H02_01,H02_02,...,MALOIM,CH_0_1_5,CH_1_5_3,D_INV_0_17,DX,V401,GL_DH_POL,GL_DH_VOZR,GL_DH_LET,GL_DH_SUPR
0,1.0,9.0,2018.0,15.0,20.0,15.0,30.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,79.0,NaN
1,1.0,9.0,2018.0,17.0,0.0,17.0,20.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,2.0,2.0,72.0,NaN
2,1.0,9.0,2018.0,15.0,35.0,15.0,55.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,89.0,NaN
3,1.0,9.0,2018.0,14.0,40.0,15.0,4.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,25.0,NaN
4,1.0,9.0,2018.0,15.0,40.0,16.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,26.0,NaN


In [26]:
factors = defaultdict(list) 
frequencuies_pandas(df, columns_to_count_frequencies, factors, 'Вся РФ')
print('===================================================================')

Наличие лифта


      Вся РФ          
        Абс.      Отн.
Нет    28422  0.473747
None   18954  0.315932
Да     12618  0.210321
Total  59994  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Вся РФ          
        Абс.      Отн.
Нет    29590  0.493216
None   18954  0.315932
Да     11450  0.190852
Total  59994  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   Вся РФ          
                                                     Абс.      Отн.
Централизованный водопровод                         51808  0.863553
Водопровод отсутствует                               3772  0.062873
Водопровод из индивидуальной артезианской скважины   2684  0.044738
Водопровод из колодца                                1710  0.028503
None                                                   20  0.000333
Total                                               59994  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Вся РФ          
                                           Абс.      Отн.
Да и имеется и используется               57925  0.965513
Нет, не имеется, не можем себе позволить    757  0.012618
Нет, не имеется, нет необходимости          503  0.008384
Да, имеется, но не используется             437  0.007284
Нет, не имею по другим причинам             352  0.005867
None                                         20  0.000333
Total                                     59994  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   Вся РФ          
                                                     Абс.      Отн.
Центральное горячее водоснабжение                   29035  0.483965
Горячее водоснабжение от местных (индивидуальны...  21043  0.350752
Горячее водоснабжение отсутствует                    9896  0.164950
None                                                   20  0.000333
Total                                               59994  1.000000
-------------------------------------------------------------------
Тип отопления в доме


                                              Вся РФ          
                                                Абс.      Отн.
Центральное отопление                          37781  0.629746
Отопление от индивидуальных установок, котлов  15801  0.263376
Печное отопление                                6259  0.104327
Другой способ отопления                          133  0.002217
None                                              20  0.000333
Total                                          59994  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Вся РФ          
                     Абс.      Отн.
Удовлетворительное  28676  0.477981
Хорошее             24720  0.412041
Отличное             3250  0.054172
Плохое               2944  0.049072
Очень плохое          404  0.006734
Total               59994  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Вся РФ          
                                               Абс.      Отн.
Отдельная квартира                            40445  0.674151
Индивидуальный (одноквартирный) дом           15532  0.258893
Часть индивидуального (одноквартирного) дома   3402  0.056706
Коммунальная квартира                           413  0.006884
Общежитие                                       184  0.003067
Другое                                           18  0.000300
Total                                         59994  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Вся РФ          
                                                     Абс.      Отн.
Да, без использования ипотечных средств или ипо...  51054  0.850985
Нет                                                  6268  0.104477
Да, с использованием ипотечных средств (ипотека...   2470  0.041171
None                                                  202  0.003367
Total                                               59994  1.000000
-------------------------------------------------------------------


In [27]:
# 2018, вся РФ, городская и сельская местность
for name in df['R_1'].unique():
    selected = df[df['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, name)
    print('===================================================================')

Наличие лифта


      городская местность          
                     Абс.      Отн.
Нет                 22675  0.544274
Да                  12405  0.297760
None                 6581  0.157965
Total               41661  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      городская местность          
                     Абс.      Отн.
Нет                 23873  0.573030
Да                  11207  0.269005
None                 6581  0.157965
Total               41661  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                                                   городская местность  \
                                                                  Абс.   
Централизованный водопровод                                      39485   
Водопровод отсутствует                                             911   
Водопровод из индивидуальной артезианской скважины                 884   
Водопровод из колодца                                              365   
None                                                                16   
Total                                                            41661   

                                                              
                                                        Отн.  
Централизованный водопровод                         0.947769  
Водопровод отсутствует                              0.021867  
Водопровод из индивидуальной артезианской скважины  0.021219  
Водопровод из колодца                               0.008761  
None                         

                                         городская местность          
                                                        Абс.      Отн.
Да и имеется и используется                            40528  0.972804
Нет, не имеется, не можем себе позволить                 406  0.009745
Нет, не имеется, нет необходимости                       277  0.006649
Да, имеется, но не используется                          238  0.005713
Нет, не имею по другим причинам                          196  0.004705
None                                                      16  0.000384
Total                                                  41661  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   сельская местность  \
                                                                 Абс.   
Горячее водоснабжение от местных (индивидуальны...               9749   
Горячее водоснабжение отсутствует                                6523   
Центральное горячее водоснабжение                                2057   
None                                                                4   
Total                                                           18333   

                                                              
                                                        Отн.  
Горячее водоснабжение от местных (индивидуальны...  0.531773  
Горячее водоснабжение отсутствует                   0.355806  
Центральное горячее водоснабжение                   0.112202  
None                                                0.000218  
Total                                               1.000000  
-----------------------------------------------

                                              сельская местность          
                                                            Абс.      Отн.
Отопление от индивидуальных установок, котлов               9609  0.524137
Печное отопление                                            4494  0.245132
Центральное отопление                                       4126  0.225059
Другой способ отопления                                      100  0.005455
None                                                           4  0.000218
Total                                                      18333  1.000000
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   сельская местность          
                                 Абс.      Отн.
Удовлетворительное               9431  0.514428
Хорошее                          6735  0.367370
Плохое                           1230  0.067092
Отличное                          790  0.043092
Очень плохое                      147  0.008018
Total                           18333  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             сельская местность          
                                                           Абс.      Отн.
Индивидуальный (одноквартирный) дом                       10142  0.553210
Отдельная квартира                                         5906  0.322151
Часть индивидуального (одноквартирного) дома               2227  0.121475
Коммунальная квартира                                        34  0.001855
Общежитие                                                    21  0.001145
Другое                                                        3  0.000164
Total                                                     18333  1.000000
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   сельская местность  \
                                                                 Абс.   
Да, без использования ипотечных средств или ипо...              16125   
Нет                                                              1852   
Да, с использованием ипотечных средств (ипотека...                332   
None                                                               24   
Total                                                           18333   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.879561  
Нет                                                 0.101020  
Да, с использованием ипотечных средств (ипотека...  0.018109  
None                                                0.001309  
Total                                               1.000000  
-----------------------------------------------

In [28]:
# 2018 год, выбираем свой регион и в нем выбираем городскую и сельскую местность
region = df[df['H00_02']=='98']
for name in region['R_1'].unique():
    print(name)
    selected = region[region['R_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1']].get(name)
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f'{region_name} {name}')
    print('===================================================================')

1.0
Наличие лифта


      Якутия городская местность          
                            Абс.      Отн.
Нет                          234  0.787879
Да                            62  0.208754
None                           1  0.003367
Total                        297  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городская местность          
                            Абс.      Отн.
Нет                          224  0.754209
Да                            72  0.242424
None                           1  0.003367
Total                        297  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городская местность          
                                                  Абс.      Отн.
Централизованный водопровод                        270  0.909091
Водопровод отсутствует                              27  0.090909
Total                                              297  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Якутия городская местность          
                                                               Абс.      Отн.
Да и имеется и используется                                     283  0.952862
Нет, не имеется, не можем себе позволить                          8  0.026936
Нет, не имею по другим причинам                                   4  0.013468
Нет, не имеется, нет необходимости                                1  0.003367
Да, имеется, но не используется                                   1  0.003367
Total                                                           297  1.000000
-------------------------------------------------------------------
Наличие холодильника


...

                                                   Якутия сельская местность  \
                                                                        Абс.   
Горячее водоснабжение отсутствует                                        177   
Горячее водоснабжение от местных (индивидуальны...                        12   
Total                                                                    189   

                                                              
                                                        Отн.  
Горячее водоснабжение отсутствует                   0.936508  
Горячее водоснабжение от местных (индивидуальны...  0.063492  
Total                                               1.000000  
-------------------------------------------------------------------
Тип отопления в доме


                                              Якутия сельская местность  \
                                                                   Абс.   
Отопление от индивидуальных установок, котлов                        80   
Центральное отопление                                                54   
Печное отопление                                                     52   
Другой способ отопления                                               3   
Total                                                               189   

                                                         
                                                   Отн.  
Отопление от индивидуальных установок, котлов  0.423280  
Центральное отопление                          0.285714  
Печное отопление                               0.275132  
Другой способ отопления                        0.015873  
Total                                          1.000000  
-------------------------------------------------------------------


                   Якутия сельская местность          
                                        Абс.      Отн.
Удовлетворительное                       106  0.560847
Хорошее                                   64  0.338624
Плохое                                    13  0.068783
Отличное                                   5  0.026455
Очень плохое                               1  0.005291
Total                                    189  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Якутия сельская местность  \
                                                                  Абс.   
Индивидуальный (одноквартирный) дом                                129   
Отдельная квартира                                                  52   
Часть индивидуального (одноквартирного) дома                         8   
Total                                                              189   

                                                        
                                                  Отн.  
Индивидуальный (одноквартирный) дом           0.682540  
Отдельная квартира                            0.275132  
Часть индивидуального (одноквартирного) дома  0.042328  
Total                                         1.000000  
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельская местность  \
                                                                        Абс.   
Да, без использования ипотечных средств или ипо...                       163   
Нет                                                                       25   
Да, с использованием ипотечных средств (ипотека...                         1   
Total                                                                    189   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.862434  
Нет                                                 0.132275  
Да, с использованием ипотечных средств (ипотека...  0.005291  
Total                                               1.000000  
-------------------------------------------------------------------


In [29]:
# 2018, в своём регионе выбираем тип города
for name in region['R_1_1'].unique():
    selected = region[region['R_1_1']==name]
    name = meta.value_labels[meta.variable_to_label['R_1_1']].get(name)
    
    frequencuies_pandas(selected, columns_to_count_frequencies, factors, f"{region_name} {name}")
    print('===================================================================')

Наличие лифта


      Якутия городской,  менее 50,0 тыс.чел          
                                       Абс.      Отн.
Нет                                     165  0.873016
Да                                       23  0.121693
None                                      1  0.005291
Total                                   189  1.000000
-------------------------------------------------------------------
Наличие мусоропровода


      Якутия городской,  менее 50,0 тыс.чел          
                                       Абс.      Отн.
Нет                                     143  0.756614
Да                                       45  0.238095
None                                      1  0.005291
Total                                   189  1.000000
-------------------------------------------------------------------
Вид водопровода в доме


                            Якутия городской,  менее 50,0 тыс.чел          
                                                             Абс.      Отн.
Централизованный водопровод                                   162  0.857143
Водопровод отсутствует                                         27  0.142857
Total                                                         189  1.000000
-------------------------------------------------------------------
Наличие стиральной машины


                                         Якутия городской,  менее 50,0 тыс.чел  \
                                                                          Абс.   
Да и имеется и используется                                                185   
Нет, не имеется, не можем себе позволить                                     3   
Нет, не имеется, нет необходимости                                           1   
Total                                                                      189   

                                                    
                                              Отн.  
Да и имеется и используется               0.978836  
Нет, не имеется, не можем себе позволить  0.015873  
Нет, не имеется, нет необходимости        0.005291  
Total                                     1.000000  
-------------------------------------------------------------------
Наличие холодильника


...

                                  Якутия сельский,  1001 - 5000 чел.     
                                                                Абс. Отн.
Горячее водоснабжение отсутствует                                 81  1.0
Total                                                             81  1.0
-------------------------------------------------------------------
Тип отопления в доме


                                              Якутия сельский,  1001 - 5000 чел.  \
                                                                            Абс.   
Центральное отопление                                                         29   
Печное отопление                                                              26   
Отопление от индивидуальных установок, котлов                                 26   
Total                                                                         81   

                                                         
                                                   Отн.  
Центральное отопление                          0.358025  
Печное отопление                               0.320988  
Отопление от индивидуальных установок, котлов  0.320988  
Total                                          1.000000  
-------------------------------------------------------------------
Оценка состояния жилого помещения


                   Якутия сельский,  1001 - 5000 чел.          
                                                 Абс.      Отн.
Удовлетворительное                                 39  0.481481
Хорошее                                            36  0.444444
Отличное                                            3  0.037037
Плохое                                              3  0.037037
Total                                              81  1.000000
-------------------------------------------------------------------
Условия проживания домохозяйства


                                             Якутия сельский,  1001 - 5000 чел.  \
                                                                           Абс.   
Индивидуальный (одноквартирный) дом                                          48   
Отдельная квартира                                                           27   
Часть индивидуального (одноквартирного) дома                                  6   
Total                                                                        81   

                                                        
                                                  Отн.  
Индивидуальный (одноквартирный) дом           0.592593  
Отдельная квартира                            0.333333  
Часть индивидуального (одноквартирного) дома  0.074074  
Total                                         1.000000  
-------------------------------------------------------------------
Является ли основное жилье собственностью домохозяйства


                                                   Якутия сельский,  1001 - 5000 чел.  \
                                                                                 Абс.   
Да, без использования ипотечных средств или ипо...                                 71   
Нет                                                                                10   
Total                                                                              81   

                                                              
                                                        Отн.  
Да, без использования ипотечных средств или ипо...  0.876543  
Нет                                                 0.123457  
Total                                               1.000000  
-------------------------------------------------------------------


In [30]:
questions = []
for question_name, answers in factors.items():
    print(question_name)
    question = pd.concat(answers, axis=1).iloc[:,:12]
    questions.append((question_name, question))
print("2018, Сельская и городская местность, вся РФ")
criteria(questions, 'сельская местность', 'городская местность', 'Вся РФ')
print(f"2018, Сельская и городская местность, {region_name}")
criteria(questions, f'{region_name} сельская местность', f'{region_name} городская местность', 'Вся РФ')

Наличие лифта
Наличие мусоропровода
Вид водопровода в доме
Наличие стиральной машины
Наличие холодильника
Пользование дома бытовым газом
Тип горячего водоснабжения
Тип отопления в доме
Оценка состояния жилого помещения
Условия проживания домохозяйства
Является ли основное жилье собственностью домохозяйства
2018, Сельская и городская местность, вся РФ
Вопрос Наличие лифта
Значение Нет
0.3134784268804887 0.5442740212668923
18333 41661
Критерий существенности различия: 54.86944714632165
--------------------------------
Значение None
0.6749031800578192 0.15796548330572957
18333 41661
Критерий существенности различия: 132.76343757760037
--------------------------------
Значение Да
0.011618393061692031 0.29776049542737815
18333 41661
Критерий существенности различия: 120.42947413209984
--------------------------------
Хи-квадрат, Вся РФ и городская местность
Power_divergenceResult(statistic=125.83482566497597, pvalue=4.734943392027245e-28)
Количество степеней свободы 2
Хи-квадрат, Вся РФ и с